In [57]:
import pandas as pd
from datetime import date, timedelta

import glicko2

In [13]:
df = pd.read_csv("bucs_results.csv")

postcodes = pd.read_csv("institute_postcodes.csv")

# print the first few rows of the DataFrame to verify that it was loaded correctly
df.head()
df = df.drop(columns=['No Score Type', 'Walkover Reason','Knockout Fixtures', 'Number of Walkovers', 'Voids', 'Blowout Game(s)', 'Zero Score?', 'Blowout to Zero', 'Team Conceding Walkover', 'Fixtures',
       'League Fixtures', 'Time'])

In [14]:
df = df[df["Fixture type"]=="League"]

In [15]:
def generate_glicko_init_rating(comp):
    if "(National)" in comp:
        return 2200
    if "North/South" in comp:
        return 1950
    if "Tier 1" in comp:
        return 1650
    if "Tier 2" in comp:
        return 1400
    if "Tier 3" in comp:
        return 1200
    if "Tier 4" in comp:
        return 1000
    if "Tier 5" in comp:
        return 800

In [16]:
df['HG_RANK_Initial'] = df.apply(lambda row: generate_glicko_init_rating(row.Competition), axis=1)
df['AG_RANK_Initial'] = df.apply(lambda row: generate_glicko_init_rating(row.Competition), axis=1)

In [17]:
df

,Community,Date,Fixture type,Competition,Home,Home Score,Away Score,Away,Winner,Venue,Team Type,Conference,Tier,Score Margin,Total Points,HG_RANK_Initial,AG_RANK_Initial
1,BUCS Hockey 19-20,25/09/2019,League,Men's Scottish Tier 3,Heriot-Watt Men's 2,0,0,Edinburgh Men's 5,NaN,Peffermill Playing Fields Provider: BUCS Hocke...,Men's,Scottish,Season,0.0,0.0,1200.0,1200.0
2,BUCS Hockey 19-20,25/09/2019,League,Men's Scottish Tier 4,Aberdeen Men's 3,0,1,Stirling Men's 3,Stirling Men's 3,"Hillhead Pitches, Hillhead Centre Provider: BU...",Men's,Scottish,Season,1.0,1.0,1000.0,1000.0
3,BUCS Hockey 19-20,25/09/2019,League,ICG Women's Scottish Tier 5,Edinburgh Women's 7,4,2,Heriot-Watt Women's 2,Edinburgh Women's 7,"Sport & Exercise Peffermill, Water Based 1 Pro...",Women's,Scottish,Season,2.0,6.0,800.0,800.0
4,BUCS Hockey 19-20,25/09/2019,League,Men's Scottish Tier 1,Aberdeen Men's 1,3,2,Heriot-Watt Men's 1,Aberdeen Men's 1,ASV Hockey Pitch (Water based) Provider: BUCS ...,Men's,Scottish,Tier 1,1.0,5.0,1650.0,1650.0
5,BUCS Hockey 19-20,25/09/2019,League,ICG Women's Scottish Tier 1,Glasgow Women's 1,0,1,Edinburgh Women's 2,Edinburgh Women's 2,"Garscube Sports Complex, West of Scotland Scie...",Women's,Scottish,Tier 1,1.0,1.0,1650.0,1650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7830,BUCS Hockey 22-23,05/10/2022,League,ICG Women's Scottish Tier 5,Abertay Women's 2,0,7,St Andrews Women's 6,St Andrews Women's 6,"DISCMains Loan, Dundee, DD4 7AA Provider: Hock...",Women's,Scottish,Season,7.0,7.0,800.0,800.0
7831,BUCS Hockey 22-23,05/10/2022,League,ICG Women's Scottish Tier 2,Heriot-Watt Women's 1,1,2,Edinburgh Women's 4,Edinburgh Women's 4,"Sport & Exercise Peffermill, Water Based 1 Pro...",Women's,Scottish,Tier 2,1.0,3.0,1400.0,1400.0
7832,BUCS Hockey 22-23,05/10/2022,League,ICG Women's Scottish Tier 2,Robert Gordon Women's 1,4,4,Aberdeen Women's 1,NaN,Countesswells Sports Field Provider: RGC,Women's,Scottish,Tier 2,0.0,8.0,1400.0,1400.0
7833,BUCS Hockey 22-23,05/10/2022,League,ICG Women's Scottish Tier 3,Glasgow Women's 3,2,0,Strathclyde Women's 2,Glasgow Women's 3,"Garscube Sports Complex, West of Scotland Scie...",Women's,Scottish,Season,2.0,2.0,1200.0,1200.0


In [53]:
team_gscores = {}

for i, team in df[df["Community"]=="BUCS Hockey 22-23"].iterrows():
    team_gscores[team.Home] = team.HG_RANK_Initial
    team_gscores[team.Away] = team.AG_RANK_Initial

print(team_gscores)

{"Cardiff Met Women's 2": 1650.0, "Cardiff Women's 2": 1650.0, "Newcastle Women's 1": 1950.0, "Loughborough Women's 2": 1950.0, "Glasgow Women's 1": 1950.0, "Nottingham Women's 2": 1950.0, "UCL Men's 5": 1000.0, "UCL Men's 4 (RUMS)": 1000.0, "Edinburgh Men's 1": 2200.0, "Nottingham Men's 1": 2200.0, "Edinburgh Women's 1": 2200.0, "Nottingham Women's 1": 2200.0, "Birmingham Men's 6": 1000.0, "Nottingham Men's 7": 1000.0, "Nottingham Trent Women's 3": 1400.0, "Oxford Brookes Women's 2": 1400.0, "Heriot-Watt Men's 2": 1200.0, "St Andrews Men's 2": 1200.0, "Liverpool Women's 2": 1400.0, "Liverpool John Moores Women's 1": 1400.0, "Edinburgh Napier Men's 1": 1200.0, "Abertay Men's 1": 1200.0, "Imperial Women's 4 (Medics)": 800.0, "St George's Women's 2": 800.0, "Arts London Women's 1": 1200.0, "St George's Women's 1": 1200.0, "Bolton Men's 1": 800.0, "Wrexham Glyndwr Men's 1": 800.0, "City Women's 1": 1000.0, "King's College Women's 5 (GKT)": 1000.0, "Nottingham Trent Women's 1": 2200.0, "Du

In [55]:
for i, row in df.iterrows():
    home_gscore = team_gscores.get(row['Home'], row['HG_RANK_Initial'])
    away_gscore = team_gscores.get(row['Away'], row['AG_RANK_Initial'])
    

1200.0 1200.0
1000.0 1000.0
800.0 800.0
1400.0 1650.0
1950.0 1650.0
1000.0 800.0
1200.0 1650.0
1650.0 1650.0
1200.0 1000.0
1650.0 1400.0
1650.0 1650.0
1000.0 1000.0
1650.0 1650.0
1400.0 1200.0
1400.0 1200.0
1400.0 1400.0
1400.0 1400.0
1000.0 1200.0
1650.0 1400.0
1400.0 1200.0
800.0 800.0
1000.0 800.0
800.0 800.0
1200.0 1400.0
800.0 800.0
800.0 800.0
1200.0 1000.0
1200.0 1200.0
800.0 800.0
1000.0 1000.0
1000.0 1000.0
1400.0 1400.0
1400.0 1400.0
800.0 1200.0
800.0 800.0
1650.0 1400.0
1200.0 1650.0
1400.0 1400.0
800.0 800.0
1000.0 1000.0
1200.0 1200.0
1650.0 1950.0
800.0 800.0
1000.0 1000.0
1200.0 1400.0
1400.0 1650.0
1650.0 1650.0
1650.0 1650.0
1650.0 1650.0
1200.0 1200.0
1000.0 1400.0
1200.0 1000.0
800.0 800.0
800.0 1000.0
1000.0 1000.0
1400.0 1200.0
1200.0 1200.0
1200.0 1650.0
1000.0 1000.0
1200.0 1200.0
1000.0 800.0
800.0 800.0
1400.0 1200.0
1000.0 1200.0
1000.0 1000.0
1000.0 1200.0
1200.0 1000.0
1000.0 1000.0
1000.0 800.0
800.0 800.0
1400.0 1400.0
1400.0 1400.0
1000.0 1000.0
1400.0 1

1400.0 1650.0
1000.0 1000.0
1400.0 1400.0
1400.0 1200.0
1200.0 1200.0
1400.0 1200.0
1200.0 1200.0
1000.0 1200.0
1650.0 1400.0
1400.0 1400.0
1200.0 1200.0
1200.0 1200.0
800.0 1000.0
1950.0 1950.0
1200.0 1400.0
1650.0 1400.0
1000.0 1000.0
800.0 800.0
1950.0 1650.0
1400.0 1400.0
800.0 800.0
1000.0 1000.0
1400.0 1000.0
1400.0 1000.0
800.0 1000.0
1650.0 1950.0
1400.0 1200.0
1950.0 2200.0
800.0 800.0
1950.0 1950.0
1200.0 1200.0
1000.0 1000.0
1000.0 1000.0
1950.0 1950.0
1000.0 1200.0
1400.0 1950.0
1200.0 1000.0
nan 1650.0
1650.0 2200.0
1950.0 1950.0
2200.0 2200.0
1950.0 1650.0
2200.0 2200.0
2200.0 1950.0
2200.0 1950.0
nan 2200.0
1950.0 1950.0
1950.0 1950.0
2200.0 2200.0
1950.0 1650.0
1950.0 1950.0
2200.0 1950.0
1000.0 1200.0
1650.0 1650.0
1400.0 1650.0
800.0 1200.0
800.0 800.0
1400.0 1400.0
1200.0 1400.0
1200.0 1000.0
1000.0 1200.0
1200.0 1000.0
1200.0 1000.0
1000.0 800.0
1200.0 1200.0
1000.0 1200.0
1000.0 800.0
1000.0 800.0
1650.0 1650.0
nan 1650.0
1400.0 1650.0
1400.0 1650.0
1400.0 1400.0
1

1000.0 800.0
2200.0 2200.0
2200.0 2200.0
1950.0 1950.0
1400.0 1650.0
800.0 1000.0
1950.0 2200.0
1950.0 1650.0
1950.0 1950.0
1400.0 1400.0
1650.0 1400.0
1400.0 1650.0
800.0 800.0
1200.0 1400.0
1950.0 nan
1950.0 1950.0
1650.0 nan
1000.0 1000.0
1950.0 1950.0
2200.0 nan
1950.0 nan
800.0 800.0
1000.0 1000.0
1400.0 1650.0
1200.0 1400.0
1950.0 1950.0
1400.0 1400.0
1400.0 1400.0
1400.0 1400.0
1000.0 1200.0
1000.0 1000.0
800.0 800.0
1000.0 1400.0
1000.0 1000.0
1400.0 1400.0
1200.0 1200.0
1000.0 1000.0
800.0 1000.0
1200.0 1400.0
1200.0 1200.0
1000.0 1200.0
1200.0 1200.0
800.0 800.0
1400.0 1650.0
1400.0 1200.0
800.0 800.0
1650.0 1650.0
1000.0 1000.0
1000.0 1000.0
1400.0 800.0
1400.0 1400.0
1000.0 800.0
800.0 800.0
800.0 800.0
1200.0 1400.0
1200.0 1200.0
1200.0 1400.0
1000.0 1000.0
1000.0 800.0
1200.0 1200.0
1200.0 1200.0
1000.0 1000.0
800.0 1400.0
1650.0 1650.0
1000.0 1000.0
1200.0 1400.0
800.0 1200.0
1400.0 1400.0
1400.0 1650.0
1000.0 1400.0
1000.0 800.0
1000.0 800.0
800.0 800.0
1200.0 1200.0
14

1200.0 1400.0
1200.0 1400.0
1200.0 1200.0
1200.0 1200.0
1400.0 1650.0
1000.0 1000.0
1200.0 1400.0
1200.0 1400.0
1400.0 1200.0
1000.0 1000.0
1200.0 1400.0
800.0 800.0
1400.0 1400.0
1200.0 1200.0
1000.0 1000.0
2200.0 1950.0
800.0 1000.0
800.0 1000.0
1650.0 1400.0
1000.0 1000.0
1000.0 1000.0
1000.0 1000.0
1200.0 1200.0
1400.0 1650.0
1200.0 1000.0
800.0 800.0
1400.0 1650.0
1650.0 1950.0
800.0 800.0
1200.0 1200.0
1000.0 1000.0
1650.0 1650.0
1650.0 1650.0
1950.0 1950.0
2200.0 2200.0
1650.0 1950.0
1950.0 1950.0
1950.0 1950.0
1650.0 1400.0
1950.0 1950.0
1650.0 1650.0
1650.0 1650.0
1400.0 1200.0
1200.0 1000.0
1950.0 1950.0
1950.0 1950.0
1950.0 1950.0
1950.0 1950.0
800.0 800.0
1400.0 1400.0
800.0 1000.0
1000.0 1000.0
800.0 800.0
800.0 800.0
1200.0 1200.0
1400.0 1400.0
1000.0 1200.0
1200.0 1200.0
1200.0 1200.0
1000.0 800.0
1650.0 1650.0
2200.0 2200.0
2200.0 1950.0
1650.0 1650.0
1200.0 1200.0
2200.0 2200.0
1200.0 1400.0
1400.0 1950.0
1650.0 1400.0
1000.0 1000.0
1400.0 1650.0
1400.0 1400.0
2200.0 2

1400.0 1400.0
1200.0 1200.0
1400.0 1200.0
1400.0 1400.0
1200.0 1200.0
1000.0 1000.0
1400.0 1200.0
1650.0 1950.0
1400.0 1400.0
1400.0 1400.0
1200.0 1200.0
1400.0 1200.0
1650.0 1950.0
1950.0 1950.0
1000.0 1000.0
1200.0 1400.0
1400.0 1200.0
800.0 800.0
1650.0 1400.0
800.0 800.0
1400.0 1400.0
1000.0 1000.0
1400.0 1400.0
1200.0 1400.0
1000.0 800.0
1650.0 1650.0
1000.0 1000.0
1400.0 1650.0
1400.0 1400.0
1400.0 1650.0
1400.0 1400.0
1400.0 1400.0
1400.0 1200.0
1650.0 1400.0
1650.0 1400.0
1650.0 1200.0
1650.0 1650.0
1400.0 1400.0
1400.0 1400.0
1000.0 1000.0
1200.0 1200.0
1000.0 1400.0
1000.0 1000.0
1650.0 1200.0
2200.0 2200.0
1200.0 1200.0
1000.0 1200.0
1950.0 1950.0
1000.0 1200.0
800.0 800.0
1000.0 1000.0
1000.0 1000.0
1400.0 1200.0
1950.0 1400.0
1200.0 1400.0
2200.0 2200.0
1950.0 1950.0
800.0 1000.0
1000.0 1200.0
1650.0 1200.0
1400.0 1650.0
1650.0 1400.0
1950.0 2200.0
1650.0 1650.0
1000.0 1200.0
1650.0 1650.0
2200.0 1950.0
1650.0 1000.0
1650.0 1650.0
2200.0 2200.0
1400.0 1200.0
800.0 800.0
12

1950.0 1950.0
1000.0 1000.0
1650.0 1650.0
1950.0 1950.0
2200.0 2200.0
1400.0 1400.0
800.0 800.0
1200.0 1200.0
1200.0 1200.0
1000.0 1000.0
1400.0 1400.0
1000.0 1000.0
1000.0 1000.0
1400.0 1400.0
1950.0 1950.0
1000.0 1000.0
1200.0 1200.0
1400.0 1400.0
1000.0 1000.0
1950.0 1950.0
1400.0 1400.0
1000.0 1000.0
2200.0 2200.0
1650.0 1650.0
1400.0 1400.0
1200.0 1200.0
1400.0 1400.0
800.0 800.0
1200.0 1200.0
1000.0 1000.0
1000.0 1000.0
1000.0 1000.0
1200.0 1200.0
1000.0 1000.0
1400.0 1400.0
1200.0 1200.0
1200.0 1200.0
1400.0 1400.0
1200.0 1200.0
1400.0 1400.0
1400.0 1400.0
1950.0 1950.0
800.0 800.0
1000.0 1000.0
1200.0 1200.0
1000.0 1000.0
1400.0 1400.0
1400.0 1400.0
2200.0 2200.0
1000.0 1000.0
1200.0 1200.0
1400.0 1400.0
1000.0 1000.0
1000.0 1000.0
1000.0 1000.0
1400.0 1400.0
800.0 800.0
1950.0 1950.0
1000.0 1000.0
1000.0 1000.0
1650.0 1650.0
1400.0 1400.0
800.0 800.0
1200.0 1200.0
1400.0 1400.0
1400.0 1400.0
1400.0 1400.0
1000.0 1000.0
1200.0 1200.0
1200.0 1200.0
1200.0 1200.0
1400.0 1400.0
10

1000.0 1000.0
1400.0 1400.0
1200.0 1200.0
1400.0 1400.0
1400.0 1400.0
1650.0 1650.0
1400.0 1400.0
1000.0 1000.0
1650.0 1650.0
1650.0 1650.0
1400.0 1400.0
1200.0 1200.0
1400.0 1400.0
800.0 800.0
800.0 800.0
1000.0 1000.0
1400.0 1400.0
1650.0 1650.0
1650.0 1650.0
1200.0 1200.0
800.0 800.0
800.0 800.0
1000.0 1000.0
1000.0 1000.0
1000.0 1000.0
1000.0 1000.0
800.0 800.0
1000.0 1000.0
1000.0 1000.0
1000.0 1000.0
1000.0 1000.0
1650.0 1650.0
1400.0 1400.0
1400.0 1400.0
1400.0 1400.0
800.0 800.0
800.0 800.0
800.0 800.0
800.0 800.0
1000.0 1000.0
1200.0 1200.0
1200.0 1200.0
1400.0 1400.0
1400.0 1400.0
1400.0 1400.0
1400.0 1400.0
1400.0 1400.0
1650.0 1650.0
1650.0 1650.0
1650.0 1650.0
1650.0 1650.0
1650.0 1650.0
1400.0 1400.0
1400.0 1400.0
1200.0 1200.0
800.0 800.0
1200.0 1200.0
1200.0 1200.0
1000.0 1000.0
1000.0 1000.0
1000.0 1000.0
1000.0 1000.0
1200.0 1200.0
1200.0 1200.0
800.0 800.0
1400.0 1400.0
1200.0 1200.0
1200.0 1200.0
1000.0 1000.0
800.0 800.0
1200.0 1200.0
1400.0 1400.0
1200.0 1200.0
12